# script for doing a continuous query of googlemaps API
it loads a list of longitude latitude coordinates and computes the time it takes to drive there by taking into account the time passed from previous segments

In [ ]:
import googlemaps
import json
import time
from datetime import datetime, timedelta

import csv
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

# here is my API key from my project
gmaps = googlemaps.Client(key='YOUR_API_KEY')

file_hin = 'routesegments.txt'
file_back = 'routesegmentsback.txt'

with open(file_hin, 'rb') as f:
    verts_hin = list(csv.reader(f))
    
with open(file_back, 'rb') as f:
    verts_back = list(csv.reader(f))
    
def compRouteDuration(verts,currtime):
    #routeArr = np.zeros(len(verts)+1);
    routeArr = [None]*(len(verts)+1)
    routeArr[0] = "{:%H:%M:%S}".format(currtime)
    routeArr[1] = 0
    currtime_new = currtime
    for ii in range(len(verts)-1):
        routeArr[ii+2] = getSegmentDuration(verts[ii],verts[ii+1],currtime_new)
        d = timedelta(seconds= routeArr[ii+2])
        currtime_new = currtime_new + d
    routeArr[1] = "{:%H:%M:%S}".format(currtime_new)
    return routeArr

def getSegmentDuration(vert1,vert2,ctime):
    distance_result = gmaps.distance_matrix(vert1[0]+','+vert1[1],
                                     vert2[0]+','+vert2[1],
                                     mode="driving",traffic_model='best_guess',
                                     departure_time=ctime)
    duration = distance_result[u'rows'][0][u'elements'][0][u'duration_in_traffic'][u'value']
    return duration



In [ ]:
strbegin = "{:%Y%B%d}".format(datetime.now())
daytimestart = '0531'
daytimestop = '1201'
cdaytime = "{:%H%M}".format(datetime.now())
#strbegin = "{:%H%M}".format(datetime.now())
f = open(strbegin+'test.txt', 'w')
for jj in range(10):
    #ctime = "{:%H%M}".format(datetime.now())
    ctime = "{:%Y%B%d}".format(datetime.now())
    cdaytime = "{:%H%M}".format(datetime.now())
    if strbegin==ctime:
        if int(daytimestart)<int(cdaytime) and int(cdaytime)<int(daytimestop):
            rarr = compRouteDuration(verts_hin,datetime.now())
            strssave = rarr[0]
            for ids in range(len(rarr)-1):
                strssave = strssave +', '+str(rarr[ids+1])
            print >> f, strssave
    else:
        f.close()
        strbegin=ctime
        f = open(strbegin+'test.txt', 'w')
        if int(daytimestart)<int(cdaytime) and int(cdaytime)<int(daytimestop):
            rarr = compRouteDuration(verts_hin,datetime.now())
            strssave = rarr[0]
            for ids in range(len(rarr)-1):
                strssave = strssave +', '+str(rarr[ids+1])
            print >> f, strssave
    time.sleep(240)
f.close()